In [27]:
""" Check if lat/lon change a lot in the files, causing the non-true-orphans problem """
import subprocess
import pandas as pd
import os
import plotly
import plotly.express as px
import plotly.graph_objects as go
#import dash  # (version 1.12.0) pip install dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import matplotlib.pylab as plt
import os,sys
import json
import xarray as xr
import numpy as np
    
import chart_studio.plotly as py
    
import numpy as np 


In [28]:
def get_orphan(d):
    csv = '../code/output_data/' + d + '_orphans.csv'  
    
    odb_dir = '/raid60/scratch/leo/scratch/era5/odbs/'
    qs='select distinct lat, lon'  #distinct,count(date) 

    
    res = { 'single' : { 'lat':[], 'lon':[], 'file':[] } ,   
            'multi'  : { 'lat':[], 'lon':[], 'file':[] } }
    
    
    lines = open(csv, 'r').readlines()
    for l in lines:
        sp = l.split(',')
        fn = sp[0]        
        
        fn = odb_dir + '/' + fn
        #print(fn)
        
        rdata=subprocess.check_output(["odb","sql","-q",qs,"-i",fn,'--no_alignment'],stderr=subprocess.DEVNULL)
        data = rdata.decode('utf-8').split('\n')
        #print(rdata)
        #print(data)
        #print(data[1:-1])

        for d in data[1:-1]:
            #print (d)
            Len = len(data[1:-1])
            if  Len ==1:
                res['single']['lat'].append(float(d.split('\t')[0]))
                res['single']['lon'].append(float(d.split('\t')[1]))
                res['single']['file'].append(fn)
            else:
                lat1, lat2 = float(data[1].split('\t')[0]) , float(data[Len-1].split('\t')[0])
                lon1, lon2 = float(data[1].split('\t')[1]) , float(data[Len-1].split('\t')[1])

                #print(lat1, lon1)
                #print(lat2, lon2)

                if abs(lat1-lat2) < 0.5 and abs(lon1-lon2) < 0.5:
                    res['single']['lat'].append(lat1)
                    res['single']['lon'].append(lon1)
                    res['single']['file'].append(fn)

                else:
                    res['multi']['lat'].append([lat1,lat2])
                    res['multi']['lon'].append([lon1,lon2])
                    res['multi']['file'].append(fn)

            
    return res

    

In [29]:
def plot(dataset, res):
    d = res['single']
    
    print(len(d['lat']))
    print(len(d['lon']))
    print(len(d['file']))

    
    df_single = pd.DataFrame.from_dict(d)

    fig = go.Figure()

    fig.add_trace( go.Scattergeo(lat=df_single.lat, 
                                 lon=df_single.lon,
                                 text = df_single.file,
                                 name = 'Orphans for ERA5 '+ dataset )
                         #hover_name="primary_id",
                         #hover_data = ['station_name'],
                         #width=1000, height=600
                 )
    
    py.iplot(fig, filename='jupyter-orphans')

    """
    fig.add_trace(go.Scattergeo (lat=miss_df.latitude, 
                                 lon=miss_df.longitude,
                                 name = 'Missing  ' + str(len(miss_df))
                         #hover_name="primary_id",
                         #hover_data = ['station_name'],
                         #width=1000, height=600
                          ) )
    """


In [ ]:
#for d in ['1','2','1759','1761','3188']:
#    a = get_orphan(d)

#for d in ['1','2','1759','1761','3188']:
ds = '1759'
res = get_orphan(ds)

d = res['single']

print(len(d['lat']))
print(len(d['lon']))
print(len(d['file']))


df_single = pd.DataFrame.from_dict(d)

fig = go.Figure()

fig.add_trace( go.Scattergeo(lat=df_single.lat, 
                             lon=df_single.lon,
                             text = df_single.file,
                             name = 'Orphans for ERA5 '+ ds )
                     #hover_name="primary_id",
                     #hover_data = ['station_name'],
                     #width=1000, height=600
             )

py.iplot(fig, filename='jupyter-orphans')